In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import rc

from ex1.visualizer import run_scenario

rc('animation', html='jshtml')
%matplotlib inline

In [ ]:
run_scenario('scenario_task_2.json', dijkstra=False).get_anim()

In [ ]:
run_scenario('scenario_task_3.json', dijkstra=False).get_anim()

In [ ]:
run_scenario('scenario_task_4_1.json', dijkstra=False).get_anim()

In [ ]:
run_scenario('scenario_task_4_1.json', dijkstra=True).get_anim()

In [ ]:
run_scenario('scenario_task_4_2.json', dijkstra=False, time_cap=30).get_anim()


In [ ]:
run_scenario('scenario_task_4_2.json', dijkstra=True).get_anim()


# Rimea


In [ ]:
run_scenario('scenario_task_5_1.json', dijkstra=True).get_anim()


In [ ]:
run_scenario('scenario_task_5_2.json', dijkstra=True).get_anim()

In [ ]:
run_scenario('scenario_task_5_3.json', dijkstra=True).get_anim()

In [ ]:
run_scenario('scenario_task_5_4.json', dijkstra=True).get_anim()

In [ ]:
run_scenario('scenario_task_5_4.json', dijkstra=True).get_anim()

In [ ]:
run_scenario('scenario_task_5_4.json', dijkstra=True).get_anim()
